In [71]:
from bs4 import BeautifulSoup
import requests
import os
import json
import pandas as pd
from wordcloud import WordCloud, ImageColorGenerator
import imageio
import seaborn as sns
import matplotlib.pyplot as plt
import jieba
import collections # 词频统计库
import time

In [2]:
url='https://www.dianping.com/searchads/ajax/suggestads'

In [38]:
form_data={
    'keyword':'花店',
    'cityId':3,
    'pageId':1,
    'userip':'122.231.171.96',
    'cookieid':'cookieid: a5b278b4-844a-3918-8818-ded22ecb382b.1589953449',
    'notAdShops':'["G57d7mdVBk7Eph8j","G6xoPbv9XvywJ6TP","lalyzFzrbTKlD91f","k8KEEA8F0rtrL6px","l2L3wHW1RIgQq3UM","k5us2LiS9CG3ULmm","G4QuOIsyXMlPLlSL","l38ypXOWS9KfKSJR","l2sAlrboYlVy9VP0","G3stpFXUU2W0Mj4c","l5tTN2j73HN9kxDC","k1M4sZcPaooG4gNI","k8BxOq6gElTXAgbd","l2ITO3nPWcbhSQKG","G3Tr7dIfvefKdpNT"]'
}

In [42]:
headers={
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
    'Referer':'Referer: https://www.dianping.com/search/keyword/3/0_%E8%8A%B1%E5%BA%97',
    'Cookie':'_lxsdk_cuid=172309c4cadc8-003c6b54d04f4f-30667d00-240000-172309c4caec8; _lxsdk=172309c4cadc8-003c6b54d04f4f-30667d00-240000-172309c4caec8; _hc.v=a5b278b4-844a-3918-8818-ded22ecb382b.1589953449; s_ViewType=10; fspop=test; cy=3; cye=hangzhou; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; Hm_lvt_602b80cf8079ae6591966cc70a3940e7=1601453372; expand=yes; _lxsdk_s=174de0f2a46-68-fa-025%7C%7C30; Hm_lpvt_602b80cf8079ae6591966cc70a3940e7=1601453388',
    'Content-Type':'application/x-www-form-urlencoded;charset=UTF-8'
}

In [43]:
res=requests.post(url,data=form_data,headers=headers)

In [44]:
res.text

'{"code":200,"msg":{"slots":[{"launches":[{"newImgUrl":"https:\\/\\/p0.meituan.net\\/dpmerchantalbum\\/2beac0a2c373692104a2c5a4f7a238a2289908.jpg","bottomprice":"300","billingtype":"1","displayTemplateId":"26","ab_trace_tag":"23-139:558,23-183:888,23-72:1490,23-127:448,23-213:1152,23-73:1584,23-142:597,20-273:1572,23-71:1423","targetid":"17526523","brandName":"下沙分店","bu":"25","regionids":"61 2083","feedback":"productid=74&pctr=0.07749677578822073&ad_cj=YJUNl7UcyXdKqulqWJ9BD_U1FlOw5x9ikFpdDtBct27PTbfmqFcZfrFi3N_NhOJD4er6aEWOhzCt3SrcwLZPww36jKH7gpKaylQQzqthQRhtKS0DXZi323kPsicGWm3eMt0ln0BwrRHM8bmJwjwmqJA_WhyRh-H_u3_DE2CGIo8&sver=2&ad_ci=YJUNn7QMxT4C-PF-AJBDHaZnVQ_usTB5nQQPBtNCvGbEQKmz-QBQOOw1jNrLlqYE9uPrdW6eimmh3SrexrFFyR6um7XenpqMgzpPmqdkFRxidy0cFtm6nH0FuXBbKlmwbt8gnQtw8BbO&dpid=cookieid: a5b278b4-844a-3918-8818-ded22ecb382b.1589953449&userid=0&bottomfen=300&page_city_id=3&entityplat=1&ad_v=2&pcvr=0.002","reviewInfo":"","discountinfo":"","productid":"74","shopId":"G992RAxu84GLKO9l","clic

In [45]:
jd=json.loads(res.text)

In [46]:
jd['msg']['slots'][0]['launches'][0]

{'newImgUrl': 'https://p0.meituan.net/dpmerchantalbum/2beac0a2c373692104a2c5a4f7a238a2289908.jpg',
 'bottomprice': '300',
 'billingtype': '1',
 'displayTemplateId': '26',
 'ab_trace_tag': '23-139:558,23-183:888,23-72:1490,23-127:448,23-213:1152,23-73:1584,23-142:597,20-273:1572,23-71:1423',
 'targetid': '17526523',
 'brandName': '下沙分店',
 'bu': '25',
 'regionids': '61 2083',
 'feedback': 'productid=74&pctr=0.07749677578822073&ad_cj=YJUNl7UcyXdKqulqWJ9BD_U1FlOw5x9ikFpdDtBct27PTbfmqFcZfrFi3N_NhOJD4er6aEWOhzCt3SrcwLZPww36jKH7gpKaylQQzqthQRhtKS0DXZi323kPsicGWm3eMt0ln0BwrRHM8bmJwjwmqJA_WhyRh-H_u3_DE2CGIo8&sver=2&ad_ci=YJUNn7QMxT4C-PF-AJBDHaZnVQ_usTB5nQQPBtNCvGbEQKmz-QBQOOw1jNrLlqYE9uPrdW6eimmh3SrexrFFyR6um7XenpqMgzpPmqdkFRxidy0cFtm6nH0FuXBbKlmwbt8gnQtw8BbO&dpid=cookieid: a5b278b4-844a-3918-8818-ded22ecb382b.1589953449&userid=0&bottomfen=300&page_city_id=3&entityplat=1&ad_v=2&pcvr=0.002',
 'reviewInfo': '',
 'discountinfo': '',
 'productid': '74',
 'shopId': 'G992RAxu84GLKO9l',
 'clickprice':

In [52]:
jd['msg']['slots'][0]['launches'][0]['shopName']

'爱情海鲜花店'

In [49]:
jd['msg']['slots'][0]['launches'][0]['regionname']

'下沙'

In [50]:
jd['msg']['slots'][0]['launches'][0]['price']

'4.4'

In [53]:
jd['msg']['slots'][0]['launches'][0]['bottomprice']

'300'

In [57]:
dataa=dict(jd['msg']['slots'][0]['launches'][0])

In [61]:
reque=[]
for i in range(8):
    reque.append(dict(jd['msg']['slots'][0]['launches'][i]))

In [62]:
reque

[{'newImgUrl': 'https://p0.meituan.net/dpmerchantalbum/2beac0a2c373692104a2c5a4f7a238a2289908.jpg',
  'bottomprice': '300',
  'billingtype': '1',
  'displayTemplateId': '26',
  'ab_trace_tag': '23-139:558,23-183:888,23-72:1490,23-127:448,23-213:1152,23-73:1584,23-142:597,20-273:1572,23-71:1423',
  'targetid': '17526523',
  'brandName': '下沙分店',
  'bu': '25',
  'regionids': '61 2083',
  'feedback': 'productid=74&pctr=0.07749677578822073&ad_cj=YJUNl7UcyXdKqulqWJ9BD_U1FlOw5x9ikFpdDtBct27PTbfmqFcZfrFi3N_NhOJD4er6aEWOhzCt3SrcwLZPww36jKH7gpKaylQQzqthQRhtKS0DXZi323kPsicGWm3eMt0ln0BwrRHM8bmJwjwmqJA_WhyRh-H_u3_DE2CGIo8&sver=2&ad_ci=YJUNn7QMxT4C-PF-AJBDHaZnVQ_usTB5nQQPBtNCvGbEQKmz-QBQOOw1jNrLlqYE9uPrdW6eimmh3SrexrFFyR6um7XenpqMgzpPmqdkFRxidy0cFtm6nH0FuXBbKlmwbt8gnQtw8BbO&dpid=cookieid: a5b278b4-844a-3918-8818-ded22ecb382b.1589953449&userid=0&bottomfen=300&page_city_id=3&entityplat=1&ad_v=2&pcvr=0.002',
  'reviewInfo': '',
  'discountinfo': '',
  'productid': '74',
  'shopId': 'G992RAxu84GLKO9l',


In [68]:
def getShopDetails(jd):
    shop=[]
    for i in range(8):
        shop.append(dict(jd['msg']['slots'][0]['launches'][i]))
       
    return shop

In [75]:
def getlink():
    url='https://www.dianping.com/searchads/ajax/suggestads'
    headers={
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
    'Referer':'Referer: https://www.dianping.com/search/keyword/3/0_%E8%8A%B1%E5%BA%97',
    'Cookie':'_lxsdk_cuid=172309c4cadc8-003c6b54d04f4f-30667d00-240000-172309c4caec8; _lxsdk=172309c4cadc8-003c6b54d04f4f-30667d00-240000-172309c4caec8; _hc.v=a5b278b4-844a-3918-8818-ded22ecb382b.1589953449; s_ViewType=10; fspop=test; cy=3; cye=hangzhou; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; Hm_lvt_602b80cf8079ae6591966cc70a3940e7=1601453372; expand=yes; _lxsdk_s=174de0f2a46-68-fa-025%7C%7C30; Hm_lpvt_602b80cf8079ae6591966cc70a3940e7=1601453388',
    'Content-Type':'application/x-www-form-urlencoded;charset=UTF-8'
}
    form_data={
    'keyword':'花店',
    'cityId':3,
    'pageId':1,
    'userip':'122.231.171.96',
    'cookieid':'cookieid: a5b278b4-844a-3918-8818-ded22ecb382b.1589953449',
    'notAdShops':'["G57d7mdVBk7Eph8j","G6xoPbv9XvywJ6TP","lalyzFzrbTKlD91f","k8KEEA8F0rtrL6px","l2L3wHW1RIgQq3UM","k5us2LiS9CG3ULmm","G4QuOIsyXMlPLlSL","l38ypXOWS9KfKSJR","l2sAlrboYlVy9VP0","G3stpFXUU2W0Mj4c","l5tTN2j73HN9kxDC","k1M4sZcPaooG4gNI","k8BxOq6gElTXAgbd","l2ITO3nPWcbhSQKG","G3Tr7dIfvefKdpNT"]'
}
    shop_total=[]
    for i in range(1,50):
        form_data['pageId']=i
        res=requests.post(url,data=form_data,headers=headers)
        jd=json.loads(res.text)
        shop_total.append(getShopDetails(jd))
        print(i)
        time.sleep(2)
    return shop_total

In [76]:
df=getlink()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [77]:
df

[[{'newImgUrl': 'https://p0.meituan.net/dpmerchantalbum/2beac0a2c373692104a2c5a4f7a238a2289908.jpg',
   'bottomprice': '300',
   'billingtype': '1',
   'displayTemplateId': '26',
   'ab_trace_tag': '23-139:558,23-183:888,23-72:1490,23-127:448,23-213:1152,23-73:1584,23-142:597,20-273:1572,23-71:1423',
   'targetid': '17526523',
   'brandName': '下沙分店',
   'bu': '25',
   'regionids': '61 2083',
   'feedback': 'productid=74&pctr=0.07749677578822073&ad_cj=PeyrGyICltqljlrlDVLtPHdtLn8CkI_bk3AJtfHTPXQO618TqF5PENh7RMgQBdy-i7GkjE3OYIPUUbsQItRjMys3HURdnP9lJjJ-pz9OEEpRyUQk6LN3_i-fA28zGXv4Ihv0Si7QvbEIyKI-sBBAbYqpIWWEANFC_bPGyy4f3-0&sver=2&ad_ci=PeyrEyMSmpPt3ELxVV3vLiQ_bSNcxqDAni5bvfXNP3oD6E1H9gMPUYYhEMQSF5j5nLi1kWbebdrYUbsSJNNpOThjClB4gPdzb1wh8zMYER1SxkUzqfJ76SfICDhqP0yWfh6hS2fXv7tf&dpid=cookieid: a5b278b4-844a-3918-8818-ded22ecb382b.1589953449&userid=0&bottomfen=300&page_city_id=3&entityplat=1&ad_v=2&pcvr=0.002',
   'reviewInfo': '',
   'discountinfo': '',
   'productid': '74',
   'shopId': 'G992R

In [79]:
dff = pd.DataFrame(df)

In [81]:
dff.to_excel('1.xlsx')

In [82]:
df=pd.read_excel('1.xlsx')

In [85]:
df[0].split(',')

AttributeError: 'Series' object has no attribute 'split'